## OpenMP CPU: Riemann sum example

In [1]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               85
Model name:          Intel(R) Xeon(R) CPU @ 2.00GHz
Stepping:            3
CPU MHz:             2000.176
BogoMIPS:            4000.35
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            1024K
L3 cache:            39424K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_si

In [2]:
%%sh 
cat > riemann_openmp_cpu.c << EOF
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <omp.h>

#define N 1000000000

double riemann(int n)
{
  double sum = 0;
  
  #pragma omp parallel for reduction(+:sum)
  for(int i = 0; i < n; ++i)
  {
    double x = (double) i / (double) n;
    sum += (exp(-x * x / 2.0) + exp(-(x + 1 / (double)n) * (x + 1 / (double)n) / 2.0)) / 2.0;
  }

  sum *= (1.0 / sqrt(2.0 * M_PI)) / (double) n;

  return sum;
}

int main(int argc, char** argv){
  double start = omp_get_wtime();
  double sum = riemann(N);

  printf("Riemann sum OpenMP CPU (double precision) for N = %d     : %.17g \n", N, sum);
  printf("Total time: \t %f s\n", omp_get_wtime()-start);
}
EOF

In [3]:
!g++ -O3 -Wall riemann_openmp_cpu.c -o riemann_openmp_cpu -fopenmp -lm

In [4]:
!OMP_NUM_THREADS=1 ./riemann_openmp_cpu

Riemann sum OpenMP CPU (double precision) for N = 1000000000     : 0.3413447460685729 
Total time: 	 38.058027 s


In [5]:
!OMP_NUM_THREADS=2 ./riemann_openmp_cpu

Riemann sum OpenMP CPU (double precision) for N = 1000000000     : 0.34134474606856957 
Total time: 	 25.856810 s
